# Hyperparameter Tuning using HyperDrive

In [ ]:
%pip install tensorflow

In [ ]:
import azureml.core
import tensorflow
print("SDK version:", azureml.core.VERSION)
print("Tensorflow version:", tensorflow.__version__)

In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
experiment_name = 'mnist-hyperdrive-experiment'
exp = Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

## Attach an Compute Instance

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# Udacity subscription does not allow to create GPU instance
cpu_instance = "mnist-instance"

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_instance)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_DS3_V2",
                                                           vm_priority='lowpriority',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_instance, compute_config)

compute_target.wait_for_completion(show_output=True)

## Dataset

The MNIST database of handwritten digits, available from this page, has a training set of 60,000 examples, and a test set of 10,000 examples. The digits have been size-normalized and centered in a fixed-size image. The images were centered in a 28x28 image by computing the center of mass of the pixels. The data is stored in a very simple file format designed for storing vectors and multidimensional matrices.

Reference: http://yann.lecun.com/exdb/mnist/

### Download the Data

In [ ]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

print("Train image shape: {}", train_images.shape)
print("Train label shape: {}", train_labels.shape)

### Transform the Data

In [ ]:
import pandas as pd

flattened_train_image = train_images.reshape(train_images.shape[0], -1)
flattened_test_image = test_images.reshape(test_images.shape[0], -1)

train_images_df = pd.DataFrame(flattened_train_image)
train_labels_series = pd.Series(train_labels, name="labels")
train_df = pd.concat([train_images_df, train_labels_series], axis=1)

test_images_df = pd.DataFrame(flattened_test_image)
test_labels_series = pd.Series(test_labels, name="labels")
test_df = pd.concat([test_images_df, test_labels_series], axis=1)

print(train_df.head())
print(test_df.head())

### Register the Data

In [ ]:
from azureml.core import Dataset

datastore = ws.get_default_datastore()
Dataset.Tabular.register_pandas_dataframe(dataframe=train_df, name='mnist-train', target=datastore)
Dataset.Tabular.register_pandas_dataframe(dataframe=test_df, name='mnist-test', target=datastore)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
from azureml.core import ScriptRunConfig, Environment
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, normal, uniform
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.policy import BanditPolicy

# # Experiment with from lower to higher dropout
# sampling = RandomParameterSampling({
#     "--dropout1": uniform(0.0, 0.1),
#     "--dropout2": uniform(0.2, 0.5),
#     "--dropout3": normal(0.3, 0.7),
#     "--epochs": choice(10, 20, 40)
# })

# # Experiment with from higher to lower dropout
# sampling = RandomParameterSampling({
#     "--dropout1": normal(0.3, 0.7),
#     "--dropout1": uniform(0.0, 0.1),
#     "--dropout3": uniform(0.2, 0.5),
#     "--epochs": choice(10, 20, 40)
# })

# Experiment with the full range of possibilites
sampling = RandomParameterSampling({
    "--dropout1": uniform(0.0, 0.5),
    "--dropout2": uniform(0.0, 0.5),
    "--epochs": choice(10, 20, 40)
})

policy = BanditPolicy(evaluation_interval=2, slack_factor=0.2)
tensorflow_env = Environment.from_conda_specification(name='tensorflow-env', file_path='conda_dependencies.yml')
script = ScriptRunConfig(source_directory='.',
                         script='mnist_train.py',
                         compute_target=compute_target,
                         environment=tensorflow_env)

hyperdrive_config = HyperDriveConfig(run_config=script,
                    hyperparameter_sampling=sampling,
                    policy=policy,
                    primary_metric_name='accuracy',
                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                    max_total_runs=10,
                    max_concurrent_runs=4)

In [ ]:
# Submit the hyperdrive run to the experiment
hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print(f"Best run: {best_run}\n")
print(f"Best run metrics: {best_run_metrics}\n")
print(f"Associated files: {best_run.get_file_names()}\n")

In [ ]:
# TODO: Download the model
# Downloading model from the run into the workspace
best_run.download_file('models/mnist_model.keras')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model = best_run.register_model(model_name='mnist_model', model_framework="TensorFlow", model_path='outputs/mnist_model.keras')

TODO: In the cell below, send a request to the web service you deployed to test it.

### Deploy an Azure Container Instance Web Service

In [ ]:
from azureml.core.model import Model
# Uncomment the following code if you want customize the deployment configuration
# from azureml.core.webservice import AciWebservice
# aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
service_name = 'mnist-service'

# Use the `deployment_config` argument for custom config
service = Model.deploy(ws, service_name, [model], overwrite=True)

service.wait_for_deployment(show_output=True)
print(service.state)

### Test the Web Service Endpoint

In [ ]:
from utils import visualize
# TODO: Replace it with the consume script provided by AML studio
# Get input data (use the dumped json from the inference script) -> Show as img -> Inference
input_data = {
    "input_key": input_value
}

# Call the web service endpoint
result = service.run(input_data)
print("Web service response:", result)

# TODO: Get the predicted labels ID
predicted_labels = ...
# TODO: Get the test labels ID
test_labels = ...

visualize[result[0]]
print("Predicted labels: ", predicted_labels[0])
print("Ground truth: ", test_labels[0])

## Delete the Resource
TODO: In the cell below, print the logs of the web service and delete the service
- webservice
- cluster

In [ ]:
service.delete()
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

